In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner, HyperbandStrategyConfig, StrategyConfig
from sagemaker.workflow.parameters import ParameterString
import json
import boto3

metric_definitions=[
    {
        "Name": "metrics/precision(B)",
        "Regex": r"YOLO Metric metrics/precision\(B\): ([0-9\.]+)"
    },
    {
        "Name": "metrics/recall(B)",
        "Regex": r"YOLO Metric metrics/recall\(B\): ([0-9\.]+)"
    },
    {
        "Name": "metrics/mAP50(B)",
        "Regex": r"YOLO Metric metrics/mAP50\(B\): ([0-9\.]+)"
    },
    {
        "Name": "metrics/mAP50-95(B)",
        "Regex": r"YOLO Metric metrics/mAP50-95\(B\): ([0-9\.]+)"
    }
]


estimator = PyTorch(entry_point='train.py',
                    role=sagemaker.get_execution_role(),
                    source_dir="./src",
                    framework_version='1.12.1',
                    py_version='py38',  # Specify the Python version here
                    instance_count=1,
                    instance_type="ml.g5.2xlarge",
                    hyperparameters={},
                    output_path='s3://{}/output'.format("hyperparameter-boundaries"))


hyperparameter_ranges = {
    'epochs': IntegerParameter(100, 1000),
    'optimizer': CategoricalParameter(['SGD', 'Adam', 'Adamax', 'AdamW', 'NAdam', 'RAdam', 'RMSProp']),
    'lr0': ContinuousParameter(0.0001, 0.01),
    'lrf': ContinuousParameter(0.0001, 0.2),
    'momentum': ContinuousParameter(0.800, 0.999),
    'weight_decay': ContinuousParameter(0.0001, 0.001),
    'warmup_epochs': ContinuousParameter(1.0, 5.0),
    'warmup_momentum': ContinuousParameter(0.500, 0.850),
    'warmup_bias_lr': ContinuousParameter(0.01, 0.1),
    'box': ContinuousParameter(0.1, 10.0),  # Box loss gain
    'cls': ContinuousParameter(0.05, 1.0),  # Class loss gain
    'hsv_h': ContinuousParameter(0.0, 0.1),  # Hue augmentation
    'hsv_s': ContinuousParameter(0.0, 1.0),  # Saturation augmentation
    'hsv_v': ContinuousParameter(0.0, 1.0),  # Value augmentation
    'degrees': ContinuousParameter(0.0, 5.0),  # Rotation degrees
    'translate': ContinuousParameter(0.0, 0.2),  # Translation factor
    'scale': ContinuousParameter(0.1, 1.0),  # Scaling factor
    'shear': ContinuousParameter(0.0, 5.0),  # Shearing degrees
    'perspective': ContinuousParameter(0.0, 0.001),  # Perspective distortion
    'flipud': ContinuousParameter(0.0, 0.5),  # Probability of flipping the image upside down
    'fliplr': ContinuousParameter(0.0, 0.5),  # Probability of flipping the image left-right
    'mosaic': ContinuousParameter(0.0, 1.0),  # Mosaic augmentation probability
    'mixup': ContinuousParameter(0.0, 1.0),  # Mixup augmentation probability
    'copy_paste': ContinuousParameter(0.0, 1.0),  # Copy-paste augmentation probability
    'auto_augment': CategoricalParameter(['randaugment', 'autoaugment', 'augmix']),
}

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name="metrics/mAP50-95(B)",
    objective_type="Maximize",
    metric_definitions=metric_definitions,
    hyperparameter_ranges=hyperparameter_ranges,
    base_tuning_job_name="boundary-segmentation",
    max_parallel_jobs=2,
    strategy='Hyperband',
    max_jobs=300,
    strategy_config=StrategyConfig(hyperband_strategy_config=HyperbandStrategyConfig(max_resource=10, min_resource=1))
)

tuner.fit({'train': 's3://boundariesdataset/Boundary_dataset/train/',
        'validation': 's3://boundariesdataset/Boundary_dataset/valid/',
          'evaluation': 's3://boundariesdataset/Boundary_dataset/test/'})

# Wait for the tuner to finish
tuner.wait()
tuning_job_name = tuner.latest_tuning_job.name
# Update the hyperparameters of the estimator with the tuning job name
estimator.set_hyperparameters(tuning_job_name=tuning_job_name)

# Get the best training job and its hyperparameters
best_training_job_name = tuner.best_training_job()
if best_training_job_name is not None:
    sagemaker_session = sagemaker.Session()
    best_job_desc = sagemaker_session.describe_training_job(best_training_job_name)
    best_hyperparameters = best_job_desc['HyperParameters']

    print("Best Hyperparameters:")
    print(best_training_job_name)
    print(best_hyperparameters)
else:
    print("No best training job found")

# # Save hyperparameters to a JSON file
# with open('best_hyperparameters.json', 'w') as f:
#     json.dump(best_hyperparameters, f)

# # Upload the file to S3
# s3 = boto3.client('s3')
# s3.upload_file('best_hyperparameters.json', 'besthyperparameter', 'parameters/best_hyperparameters.json')



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


Using provided s3_resource
.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................